# answers 09: the adventure of the ten Arcs
**Sean's answer**

## 1. Write a simulator as a positive control

> Write a script that simulates N=1000000 observed read counts, following the model specified above, 
> for any Arc locus structure (i.e. lengths $L_t$) and any transcript abundances $\tau$. (You can assume that 
> there are 10 segments and isoforms, though my version of this script will allow that to vary too.)
> Use your script to generate a test data set for known model parameters $\tau,L$
> that you’ve chosen.

**The point I want you to get from this: when you are thinking of data analysis in terms of a generative probability model of the data, you automatically have a way to generate positive control data: sample synthetic data from your model.**



In [1]:
import numpy as np
import string
import sys

I'll write my generator so it takes two controlling arguments: a random number seed (so I can generate the same data reproducibly from run to run, for a given seed), and a flag for whether to use the pset's Arc transcript lengths, or to generate new ones. I have them at the top of my notebook, so I can change them and rerun the page.

I also set up some other parameters of the problem here: the number of Arc segments, and the number of reads to generate.

In [2]:
rng_seed  = 2020
do_pset_L = False

S         = 10        # Number of segments in the Arc locus 
T         = 10        # Number of different transcripts T (is the same, one starting on each segment)   
N         = 1000000   # total number of observed reads to generate: \sum_k r_s

Slabel  = list(string.ascii_uppercase)[:S]               # ['A'..'J']        : the upper case labels for Arc locus segments 
Tlabel  = [ "Arc{}".format(d) for d in range(1,T+1) ]    # ['Arc1'..'Arc10'] : the labels for Arc transcript isoforms
Rlabel  = list(string.ascii_lowercase)[:T]               # ['a'..'j']        : lower case labels for reads

A "structure" of an Arc locus is defined by T and by an array of lengths of each Arc transcript. 

In [3]:
def generate_Arc_structure(do_pset_L, T):  
    if do_pset_L:
        assert(T == 10)
        L    = np.array([ 4, 2, 3, 4, 4, 3, 2, 2, 3, 3 ])    # lengths matching the pset figure
    else:
        L    = [ np.random.randint(2,5) for _ in range(T) ]  # or sampling: each isoform t is 2..4 segments long; P(S|T) = 1/L_t
    return L

We can make our life easier at some steps by defining some auxiliary data, dependent on the structure of the Arc problem. `seg_is_in[t][s]` is a Kronecker delta function evaluating to 1 if segment s is in transcript t, else 0. This gets used in the EM algorithm and the log likelihood calculation. `segusage[s]` is the number of Arc transcripts that contain segment s. This gets used in Lestrade's naive solution.

In [4]:
def make_auxiliary_arrays(L, T, S):
    seg_is_in = np.zeros((T,S)).astype(int)
    segusage  = np.zeros(S).astype(int)
 
    for t in range(T):
        for s in range(t,t+L[t]):
            seg_is_in[t][s%S] = 1
            segusage[s%S]    += 1
            
    return seg_is_in, segusage


The pset dictates that expression levels are in transcript abundance ($\tau$) in input and output, but we need them to be in nucleotide abundance ($\nu$) for the generative model. Here's a couple of functions to convert back and forth.

In [5]:
def to_nu(tau, L):
    nu = np.multiply(tau, L)              
    nu = np.divide(nu, np.sum(nu))
    return nu

def to_tau(nu, L):
    tau = np.divide(nu, L)              
    tau = np.divide(tau, np.sum(tau))
    return tau

A test dataset consists of sampling $\tau$, then using those known $\tau$ to sample counts $r$.

In [6]:
# Sample known tau, then generate the observed "read" counts, r
# The generative model says there's two steps:
#   P(T)     = nu    : probability of selecting isoform t (~ expression level of t, in *nucleotides*)
#   P(S | T) = 1/L_t : probability of selecting segment s in transcript t
# So we just go through these two steps.
#
def generate_test_data(rng_seed, T, L, N):
    np.random.seed(rng_seed)
    
    tau = np.random.dirichlet(np.ones(T))      # true transcript abundances \tau
    nu  = to_nu(tau, L)

    r = np.zeros(S).astype(int)
    for n in range(N):                         # for each independent read that we sample:
        t = np.random.choice(T, p=nu)          # choice of isoform t = 0..T-1, using \nu expression levels
        s = (t + np.random.choice(L[t])) % S   # choice of segment s = 0..L[t]-1 in isoform, using 1/L[t] uniform distribution
        r[s] += 1                              # count one mapped read, 0..S-1
        
    return(tau, r)

OK, that's everything we need (and more) for part 1. Now use these functions to generate a test data set.

In [7]:
L                   = generate_Arc_structure(do_pset_L, T)
true_tau, r         = generate_test_data(rng_seed, T, L, N)
true_nu             = to_nu(true_tau, L)
seg_is_in, segusage = make_auxiliary_arrays(L, T, S)

In [8]:
L

[2, 4, 3, 3, 2, 2, 2, 3, 4, 2]

In [9]:
true_tau

array([0.39212232, 0.18895911, 0.06517565, 0.02900489, 0.03756561,
       0.02236103, 0.02958956, 0.03845195, 0.1811869 , 0.01558298])

In [10]:
r

array([204842, 264696,  88979,  98645, 111545,  30982,  18147,  24019,
        76691,  81454])

## 2. Calculate the log likelihood

We're also asked to use Lestrade's script to estimate parameters by their method. On our test data, we calculate the log likelihood of the true parameters versus Lestrade's parameters. We're going to see that Lestrade's solution is crappy, as you probably guessed already.

We could run Lestrade's script externally, but let's pull in its code here as a function.

**A point I'm emphasizing with this: if we're trying to estimate unknown parameters of a model, one standard thing to do is to find parameters that maximize the likelihood of our model: "ML" parameter estimates.** It's very typical to have a function that calculates the total probability $\log P(D \mid \theta)$ of your observed data $D$ and some current model parameters $\theta$. There are various ways to optimize $\theta$. Sometimes we'll directly optimize the parameters, either analytically or numerically. Problems where we'd use expectation maximization to find ML parameters arise when we have additional latent random variables whose values we're not interested in, but which we need to know to estimate our parameters. Here, we're missing the true assignment of read counts r to isoforms T.

In [11]:
# Define a function that calculates the total log likelihood of the
# observed data, given nucleotide abundances nu.
#
#   P(S) = \sum_T P(S,T) = \sum_T P(S | T) P(T) =  \sum_t (\nu_t / L_t)  \delta(S is in T=t)
#
def loglikelihood(r, nu, L, T, S, seg_is_in):
    """ 
    Function: loglikelihood()  
    Calculates the likelihood of the observed read counts r_1..r_S, 
    given current expression level parameters nu_1..nu_T. 
    Leaves off the multinomial coefficient, because it's a constant
    that depends only on the given data, not on the abundance parameters.
    """  
    logL     = 0
    for s in range(S):                         # Likelihood of getting r_s observed counts for each read sequence a..j 
        p = 0.
        for t in range(T):                     # ... summed over possible transcripts Ti
            if seg_is_in[t][s]:
                p += nu[t] / L[t]              # P(S) = \sum_T P(T,S) 
        logL += r[s] * np.log(p)
    return logL

In [12]:
def lestrade_estimate(r, L, T, S, segusage):
    """
    Function: lestrade_estimate()
    Lestrade et al. simply assign each read to each isoform it could have come from,
    equally distributing the one read count, ignoring the read accuracy.
    """
    c  = np.zeros(T)
    for t in range(T):
        for s in range(t,t+L[t]):
            c[t] += (1.0 / float(segusage[s%S])) * float(r[s%S])  # For each read s, assign 1/usage count to each transcript
                                                                  # that contains segment j.
    Z      = np.sum(c)
    est_nu = np.divide(c, Z)
    return est_nu

Now we can run Lestrade's method on our generated test data.

In [13]:
nu_lestrade  = lestrade_estimate(r, L, T, S, segusage)
tau_lestrade = to_tau(nu_lestrade, L)                     

# Print a table of the resulting estimates for tau
print("Lestrade's estimates:")
print("{0:10s}  {1:>9s}  {2:>9s}".format("isoform", "est_tau", "true_tau"))
for i in range(T):
    print ("{0:10s}  {1:9.3f}  {2:9.3f}".format(Tlabel[i], tau_lestrade[i], true_tau[i]))
print('')

print("logL, est  = {0:.1f}".format(loglikelihood(r, nu_lestrade, L, T, S, seg_is_in)))
print("logL, true = {0:.1f}".format(loglikelihood(r, true_nu,     L, T, S, seg_is_in)))

Lestrade's estimates:
isoform       est_tau   true_tau
Arc1            0.221      0.392
Arc2            0.137      0.189
Arc3            0.099      0.065
Arc4            0.067      0.029
Arc5            0.054      0.038
Arc6            0.027      0.022
Arc7            0.030      0.030
Arc8            0.073      0.038
Arc9            0.157      0.181
Arc10           0.135      0.016

logL, est  = -2073371.9
logL, true = -2036190.9


The Lestrade solution has a worse (more negative) log likelihood than the true parameters.

## 3. estimate isoform abundances by EM

> Write a script that estimates unknown isoform abundances $\tau_i$ for each isoform Arc1..Arc10, 
> given read counts $r_k$ and the structure of the Arc locus including the lengths $L_i$,
> using expectation maximization.
> Apply your script to the data in the Lestrade et al. supplementary data file. Show your estimated $\tau_i$.

We could parse the Lestrade supplementary data file - grabbing it through `urllib` even - and Lestrade's
own script provided the parsing code we could use, but it's easy enough to just drop its data in here.



In [14]:
# Analysis by expectation maximization.
#
# If you told me nu[]; then I could tell you the expected number of
# reads mapping to isoform i, c[t], because you can calculate P(T|S).
# (Expectation step.)
#
# If you told me c[]; then I could tell you the maximum likelihood
# estimate of nu[], which is simply the normalized counts. 
# (Maximization step.)
#
# But I tell you neither; I only tell you r[], the observed reads.
#
def expectation_maximization(r, L, T, S, seg_is_in):
    est_nu      = np.random.dirichlet(np.ones(T))   # random starting guess.
    converged   = False
    iterations  = 0
    while not converged:
        iterations += 1
    
        # Expectation step. 
        # We need P(T | S): expected prob that a read mapped to S came from transcript T.
        #   P(T|S) =  P(T,S) / \sum_T P(T,S) = nu_i * 1/L_i if segment S is in T, normalize over T. 
        c = np.zeros(T)
        for s in range(S):      # For a given choice of read S, where we've observed r_s counts
            Pr = np.zeros(T)    #   ... we're going to calculate P(T|S)
            for t in range(T):  #     ... by first calculating the numerator P(T,S) = \nu_i/L_i for the S in T
                if seg_is_in[t][s]:
                    Pr[t] +=  est_nu[t] / float(L[t])  
            Z  = np.sum(Pr)            # ... now Z = P(S) 
            Pr = np.divide(Pr, Z)      # ... normalizing gives us P(T | S) = P(S,T) / P(S), for this read s

            # Now <Pr> is P(T=t | S=s), for read s; we dole out the observed counts
            # r[s] to the possible transcripts they came
            # from, proportional to the probability they came from that one.
            #
            for t in range(T):
                c[t] += Pr[t] * r[s]

        # Maximization step
        # ML estimate for nu is just the frequency.
        Z      = np.sum(c)
        est_nu = np.divide(c, Z)

        # We could make a fancy convergence test here.
        # Running for 1000 iterations suffices, though it's overkill.
        if iterations == 1000: converged = True
    return est_nu

Analyze our test data set by EM.

In [15]:
est_nu  = expectation_maximization(r, L, T, S, seg_is_in)
est_tau = to_tau(est_nu, L)

In [16]:
print("EM estimates:")
print("{0:10s}  {1:>9s}  {2:>9s}".format("isoform", "est_tau", "true_tau"))
for i in range(T):
    print ("{0:10s}  {1:9.3f}  {2:9.3f}".format(Tlabel[i], est_tau[i], true_tau[i]))
print('')

print("logL, est  = {0:.1f}".format(loglikelihood(r, est_nu,  L, T, S, seg_is_in)))
print("logL, true = {0:.1f}".format(loglikelihood(r, true_nu, L, T, S, seg_is_in)))

EM estimates:
isoform       est_tau   true_tau
Arc1            0.396      0.392
Arc2            0.184      0.189
Arc3            0.071      0.065
Arc4            0.027      0.029
Arc5            0.036      0.038
Arc6            0.026      0.022
Arc7            0.026      0.030
Arc8            0.043      0.038
Arc9            0.177      0.181
Arc10           0.014      0.016

logL, est  = -2036185.4
logL, true = -2036190.9


Looks ok. Our estimated $\tau$ are almost identical to the known true $\tau$ on our generated data: a positive control that our EM estimation is working properly.  Our log likelihood is guaranteed to be a global optimum in this problem, so another check that things are working as expected is that we obtain a log likelihood at least as good as the likelihood of the true $\tau$, and we do. We do a little better because we're overfitting stochastic noise in the data to some negligible degree.

We're ready to analyze Lestrade's supplementary data file now!

We could parse the data file but it's easy enough to just plug the numbers in:

In [17]:
# The supplementary data from Lestrade et al.
T = 10
S = 10


r = np.array([  89357,  69196, 87852, 99854, 57369,  85715, 197730, 213016, 61271, 38640 ]).astype(int)
L = np.array([      4,      2,     3,     4,     4,      3,      2,      2,     3,     3 ]).astype(int)

seg_is_in, segusage = make_auxiliary_arrays(L, T, S)

The solution that Lestrade got:

In [18]:
nu_lestrade  = lestrade_estimate(r, L, T, S, segusage)
tau_lestrade = to_tau(nu_lestrade, L)                     

Our EM solution:

In [19]:
est_nu  = expectation_maximization(r, L, T, S, seg_is_in)
est_tau = to_tau(est_nu, L)

In [20]:
print("{0:10s}  {1:>9s}   {2:>9s}".format("isoform", "Lestrade", "ours"))
for i in range(T):
    print ("{0:10s}  {1:9.3f}   {2:9.3f}".format(Tlabel[i], tau_lestrade[i], est_tau[i]))
print('')

print("logL, Lestrade's  = {0:.1f}".format(loglikelihood(r, nu_lestrade, L, T, S, seg_is_in)))
print("logL, ours        = {0:.1f}".format(loglikelihood(r, est_nu,      L, T, S, seg_is_in)))


isoform      Lestrade        ours
Arc1            0.085       0.142
Arc2            0.077       0.019
Arc3            0.080       0.084
Arc4            0.096       0.053
Arc5            0.111       0.023
Arc6            0.129       0.163
Arc7            0.151       0.313
Arc8            0.123       0.095
Arc9            0.078       0.076
Arc10           0.071       0.032

logL, Lestrade's  = -2190089.6
logL, ours        = -2165610.4


**that's it...** 
When we analyze Lestrade's data, we find that their conclusion that all Arc isoforms are roughly equally expressed is wrong. Their analysis biased their conclusion, by assigning read counts uniformly to the possible isoforms they could have come from. Actually, Arc2, Arc5, and Arc10 are only weakly expressed, and Arc7 accounts for 30% of the RNA population; the difference between their expression levels is about 10-15x.